<a href="https://colab.research.google.com/github/deepmind/open_spiel/blob/master/open_spiel/colabs/Open_Spiel_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

Install a recent clang compiler:

In [ ]:
!apt-get install clang-9

Build & install the pip source distribution of OpenSpiel:

In [ ]:
!CXX=`which clang++-9` pip install --upgrade --verbose open_spiel

# It's play time!

In [ ]:
import itertools as it

import numpy as np
from matplotlib import pyplot as plt
import pyspiel

np.set_printoptions(precision=3, suppress=True, floatmode='fixed')

from open_spiel.python.algorithms import exploitability
from open_spiel.python import policy as policy_lib

game = pyspiel.load_game('tic_tac_toe')
state = game.new_initial_state()

while not state.is_terminal():
  action = np.random.choice(state.legal_actions())
  print(f'Taking action {action} {state.action_to_string(action)}')
  state.apply_action(action)
  print(state)
print(f'Game over; returns {state.returns()}')

In [ ]:
game = pyspiel.load_game('kuhn_poker')
print(game.get_type().pretty_print())

In [ ]:
policy = policy_lib.TabularPolicy(game)
print(policy.states_per_player)
print(policy.action_probability_array)

In [ ]:
def print_policy(policy):
  for state, probs in zip(it.chain(*policy.states_per_player),
                          policy.action_probability_array):
    print(f'{state:6}   p={probs}')

print_policy(policy)

In [ ]:
print(exploitability.nash_conv(game, policy))

In [ ]:
def new_reach(so_far, player, action_prob):
  """Returns new reach probabilities."""
  new = np.array(so_far)
  new[player] *= action_prob
  return new

def calc_cfr(state, reach):
  """Updates regrets; returns utility for all players."""
  if state.is_terminal():
    return state.returns()
  elif state.is_chance_node():
    return sum(prob * calc_cfr(state.child(action), new_reach(reach, -1, prob))
               for action, prob in state.chance_outcomes())
  else:
    # We are at a player decision point.
    player = state.current_player()
    index = policy.state_index(state)
    
    # Compute utilities after each action, updating regrets deeper in the tree.
    utility = np.zeros((game.num_distinct_actions(), game.num_players()))
    for action in state.legal_actions():
      prob = curr_policy[index][action]
      utility[action] = calc_cfr(state.child(action), new_reach(reach, player, prob))

    # Compute regrets at this state.
    cfr_prob = np.prod(reach[:player]) * np.prod(reach[player+1:])
    value = np.einsum('ap,a->p', utility, curr_policy[index])
    for action in state.legal_actions():
      regrets[index][action] += cfr_prob * (utility[action][player] - value[player])

    # Return the value of this state for all players.
    return value

In [ ]:
game = pyspiel.load_game('kuhn_poker')
policy = policy_lib.TabularPolicy(game)
initial_state = game.new_initial_state()
curr_policy = policy.action_probability_array.copy()
regrets = np.zeros_like(policy.action_probability_array)
eval_steps = []
eval_nash_conv = []
for step in range(2049):
  # Compute regrets
  calc_cfr(initial_state, np.ones(1 + game.num_players()))

  # Find the new regret-matching policy
  floored_regrets = np.maximum(regrets, 1e-16)
  sum_floored_regrets = np.sum(floored_regrets, axis=1, keepdims=True)
  curr_policy = floored_regrets / sum_floored_regrets

  # Update the average policy
  lr = 1 / (1 + step)
  policy.action_probability_array *= (1 - lr)
  policy.action_probability_array += curr_policy * lr

  # Evaluate the average policy
  if step & (step-1) == 0:
    nc = exploitability.nash_conv(game, policy)
    eval_steps.append(step)
    eval_nash_conv.append(nc)
    print(f'Nash conv after step {step} is {nc}')

In [ ]:
fig, ax = plt.subplots()
ax.set_title("NashConv by CFR Iteration")
ax.plot(eval_steps, eval_nash_conv)
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("NashConv by CFR Iteration (log-log scale)")
ax.loglog(eval_steps, eval_nash_conv)
fig.show()

In [ ]:
# Display the whole policy
print_policy(policy)

# How likely are we to bet with a Jack?
alpha = policy.action_probability_array[policy.state_lookup['0']][1]
print(f'P(bet with Jack) = alpha = {alpha:.3}')

# How likely are we to bet with a King?
pK = policy.action_probability_array[policy.state_lookup['2']][1]
print(f'P(bet with King) = {pK:.3}, cf {alpha * 3:.3}')

# How likely are we to call with a Queen?
pQ = policy.action_probability_array[policy.state_lookup['1pb']][1]
print(f'P(call with Queen after checking) = {pQ:.3}, cf {alpha + 1/3:.3}')


In [ ]:
def sample(actions_and_probs):
  actions, probs = zip(*actions_and_probs)
  return np.random.choice(actions, p=probs)

def policy_as_list(policy, state):
  return list(enumerate(policy.policy_for_key(state.information_state_string())))

def env_action(state):
  if state.is_chance_node():
    p = state.chance_outcomes()
  else:
    p = policy_as_list(fixed_policy, state)
  return sample(p)

def softmax(x):
  x = np.exp(x - np.max(x, axis=-1, keepdims=True))
  return x / np.sum(x, axis=-1, keepdims=True)

def generate_trajectory(state, player):
  trajectory = []
  while not state.is_terminal():
    if state.current_player() == player:
      action = sample(policy_as_list(rl_policy, state))
      trajectory.append((rl_policy.state_index(state), action))
    else:
      action = env_action(state)
    state.apply_action(action)
  return trajectory, state.returns()[player]


In [ ]:
fixed_policy = policy_lib.TabularPolicy(game)
rl_policy = policy_lib.TabularPolicy(game)
for _ in range(5):
  print(generate_trajectory(game.new_initial_state(), player=0))

In [ ]:
# Run REINFORCE
N = 10000
lr = 0.01
for step in range(N):
  for player in (0, 1):
    trajectory, reward = generate_trajectory(game.new_initial_state(), player)
    for s, a in trajectory:
      logits = np.log(rl_policy.action_probability_array[s])
      logits[a] += lr * reward
      rl_policy.action_probability_array[s] = softmax(logits)

In [ ]:
# Evaluate the policy
def evaluate(state, rl_policy, player):
  if state.is_terminal():
    return state.returns()[player]
  elif state.current_player() == player:
    ap = policy_as_list(rl_policy, state)
  elif state.is_chance_node():
    ap = state.chance_outcomes()
  else:
    ap = policy_as_list(fixed_policy, state)
  return sum(p * evaluate(state.child(a), rl_policy, player) for a, p in ap)
  
def eval(rl_policy):
  return (evaluate(game.new_initial_state(), rl_policy, player=0)
        + evaluate(game.new_initial_state(), rl_policy, player=1))

print_policy(rl_policy)  
eval(rl_policy)

In [ ]:
# Evaluate the greedy policy
greedy_policy = policy_lib.TabularPolicy(game)
greedy_policy.action_probability_array = (np.eye(game.num_distinct_actions())
              [np.argmax(rl_policy.action_probability_array, axis=-1)])

print_policy(greedy_policy)
eval(greedy_policy)